<big><big><b>PyCity Schools Analysis</b></big></big>  <br>

OBSERVED TREND 1:  Charter schools perform better than district shcools  <br>
OBSERVED TREND 2:  The performance of a school's students is inversely related to the size of the school.  <br>
OBSERVED TREND 3:  The data suggests that the performance of a school's students is inversley related to the school's spending per student.  However, it's possible that this is simply because larger shcools are more expensive, and larger schools perform worse.  <br>

In [1]:
import pandas as pd

schools_inpath = 'raw_data/schools_complete.csv'
students_inpath = 'raw_data/students_complete.csv'

schools_df = pd.read_csv(schools_inpath, dtype={'School ID' : int, 
                                                'name' : object, 
                                                'type' : object, 
                                                'size' : int, 
                                                'budget' : int})

students_df = pd.read_csv(students_inpath, dtype = {'Student ID' : int,
                                                    'name' : object,
                                                    'gender' : object,
                                                    'grade' : object,
                                                    'school' : object,
                                                    'reading_score' : int,
                                                    'math_score' : int})


<big><b>District Summary</b></big>

In [2]:
total_schools = schools_df['name'].count()
total_students = students_df['name'].count()
total_budget = schools_df['budget'].sum()
average_math_score = students_df['math_score'].sum() / total_students
average_reading_score = students_df['reading_score'].sum() / total_students

passing_students_math = students_df.loc[students_df['math_score'] > 60, :]['name'].count()
passing_rate_math = (passing_students_math / total_students) * 100

passing_students_reading = students_df.loc[students_df['reading_score'] > 60, :]['name'].count()
passing_rate_reading = (passing_students_reading / total_students) * 100

overall_passing_rate = (passing_rate_math + passing_rate_reading) / 2

district_summary_df = pd.DataFrame(
        {
            'Total Schools' : total_schools,
            'Total Students' : total_students,
            'Total Budget' : total_budget,
            'Average Math Score' : average_math_score,
            'Average Reading Score' : average_reading_score,
            '% Passing Math' : passing_rate_math,
            '% Passing Reading' : passing_rate_reading,
            '% Overall Passing Rate' : overall_passing_rate
        },
    index = [0]
)

district_summary_df['Total Budget'] = district_summary_df['Total Budget'].map('$ {:,.2f}'.format)

district_summary_df = district_summary_df[['Total Schools', 'Total Students', 'Total Budget', 
                                           'Average Math Score', 'Average Reading Score', 
                                           '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$ 24,649,428.00",78.985371,81.87784,90.906306,100.0,95.453153


<big><b>School Summary</b></big>

In [3]:
total_students_school = students_df.groupby('school')['name'].count()
total_budget_school = schools_df.groupby('name')['budget'].sum()

avg_math_score_school = students_df.groupby('school')['math_score'].sum() / total_students_school
avg_reading_score_school = students_df.groupby('school')['reading_score'].sum() / total_students_school

passing_math_school = students_df.loc[students_df['math_score'] > 60, :].groupby('school')['math_score'].count()
passing_reading_school = students_df.loc[students_df['reading_score'] > 60, :].groupby('school')['reading_score'].count()

schools_df_addendum = pd.concat([total_students_school, 
                                 total_budget_school, 
                                 avg_math_score_school, 
                                 avg_reading_score_school, 
                                 passing_math_school, 
                                 passing_reading_school], 
                               axis = 1)

schools_df_addendum.reset_index(inplace=True)

schools_df_addendum = schools_df_addendum.rename(columns={'index' : 'name',  
                                                        'name' : 'Total Students',
                                                        'budget' : 'Total School Budget',
                                                        0 : 'Average Math Score',
                                                        1 : 'Average Reading Score',
                                                        'math_score' : 'passing math raw',
                                                        'reading_score' : 'passing reading raw'
                                                        })

schools_df_addendum = schools_df.merge(schools_df_addendum, on='name')

schools_df_addendum['Per Student Budget'] = schools_df_addendum['Total School Budget'] / schools_df_addendum['Total Students']
schools_df_addendum['% Passing Math'] = (schools_df_addendum['passing math raw'] / schools_df_addendum['Total Students']) * 100
schools_df_addendum['% Passing Reading'] = (schools_df_addendum['passing reading raw'] / schools_df_addendum['Total Students']) * 100
schools_df_addendum['% Overall Passing Rate'] = (schools_df_addendum['% Passing Math'] + schools_df_addendum['% Passing Reading']) / 2

schools_summary = schools_df_addendum[['name', 
                                       'type', 
                                       'Total Students', 
                                       'Total School Budget', 
                                       'Per Student Budget', 
                                       'Average Math Score', 
                                       'Average Reading Score', 
                                       '% Passing Math', 
                                       '% Passing Reading', 
                                       '% Overall Passing Rate']]
                                       
schools_summary = schools_summary.rename(columns={'type' : 'School Type'})
schools_summary.set_index('name', inplace=True)
del schools_summary.index.name

schools_summary['Total School Budget'] = schools_summary['Total School Budget'].map('$ {:,.2f}'.format)
schools_summary['Per Student Budget'] = schools_summary['Per Student Budget'].map('$ {:,.2f}'.format)


schools_summary


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Huang High School,District,2917,"$ 1,910,635.00",$ 655.00,76.629414,81.182722,86.835790,100.0,93.417895
Figueroa High School,District,2949,"$ 1,884,411.00",$ 639.00,76.711767,81.158020,86.436080,100.0,93.218040
Shelton High School,Charter,1761,"$ 1,056,600.00",$ 600.00,83.359455,83.725724,100.000000,100.0,100.000000
Hernandez High School,District,4635,"$ 3,022,020.00",$ 652.00,77.289752,80.934412,86.450917,100.0,93.225458
Griffin High School,Charter,1468,"$ 917,500.00",$ 625.00,83.351499,83.816757,100.000000,100.0,100.000000
Wilson High School,Charter,2283,"$ 1,319,574.00",$ 578.00,83.274201,83.989488,100.000000,100.0,100.000000
Cabrera High School,Charter,1858,"$ 1,081,356.00",$ 582.00,83.061895,83.975780,100.000000,100.0,100.000000
Bailey High School,District,4976,"$ 3,124,928.00",$ 628.00,77.048432,81.033963,87.439711,100.0,93.719855
Holden High School,Charter,427,"$ 248,087.00",$ 581.00,83.803279,83.814988,100.000000,100.0,100.000000
Pena High School,Charter,962,"$ 585,858.00",$ 609.00,83.839917,84.044699,100.000000,100.0,100.000000


<big><b>Top Performing Schools (By Passing Rate)</b></big>

In [4]:
#top_performing_schools = schools_summary.sort_values(['% Overall Passing Rate', 'Per Student Budget'], ascending=[False, True]).iloc[0:5,]
top_performing_schools = schools_summary.sort_values('% Overall Passing Rate', ascending=False).iloc[0:5,]
top_performing_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Shelton High School,Charter,1761,"$ 1,056,600.00",$ 600.00,83.359455,83.725724,100.0,100.0,100.0
Griffin High School,Charter,1468,"$ 917,500.00",$ 625.00,83.351499,83.816757,100.0,100.0,100.0
Wilson High School,Charter,2283,"$ 1,319,574.00",$ 578.00,83.274201,83.989488,100.0,100.0,100.0
Cabrera High School,Charter,1858,"$ 1,081,356.00",$ 582.00,83.061895,83.975780,100.0,100.0,100.0
Holden High School,Charter,427,"$ 248,087.00",$ 581.00,83.803279,83.814988,100.0,100.0,100.0


<big><b>Bottom Performing Schools (By Passing Rate)</b></big>

In [5]:
#bottom_performing_schools = schools_summary.sort_values(['% Overall Passing Rate', 'Per Student Budget'], ascending=[True, False]).iloc[0:5,]
bottom_performing_schools = schools_summary.sort_values('% Overall Passing Rate').iloc[0:5,]
bottom_performing_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Figueroa High School,District,2949,"$ 1,884,411.00",$ 639.00,76.711767,81.158020,86.436080,100.0,93.218040
Rodriguez High School,District,3999,"$ 2,547,363.00",$ 637.00,76.842711,80.744686,86.446612,100.0,93.223306
Hernandez High School,District,4635,"$ 3,022,020.00",$ 652.00,77.289752,80.934412,86.450917,100.0,93.225458
Johnson High School,District,4761,"$ 3,094,650.00",$ 650.00,77.072464,80.966394,86.704474,100.0,93.352237
Huang High School,District,2917,"$ 1,910,635.00",$ 655.00,76.629414,81.182722,86.835790,100.0,93.417895


<big><b>Math Scores by Grade</b></big>

In [6]:
students_9  = students_df.loc[students_df['grade'] == '9th',]
students_10 = students_df.loc[students_df['grade'] == '10th',]
students_11 = students_df.loc[students_df['grade'] == '11th',]
students_12 = students_df.loc[students_df['grade'] == '12th',]

students_9_school_count  = students_9.groupby('school')['grade'].count()
students_10_school_count = students_10.groupby('school')['grade'].count()
students_11_school_count = students_11.groupby('school')['grade'].count()
students_12_school_count = students_12.groupby('school')['grade'].count()

math_9_school  = students_9.groupby('school')['math_score'].sum() / students_9_school_count
math_10_school = students_10.groupby('school')['math_score'].sum() / students_10_school_count
math_11_school = students_11.groupby('school')['math_score'].sum() / students_11_school_count
math_12_school = students_12.groupby('school')['math_score'].sum() / students_12_school_count

math_scores_grade_school = pd.concat([math_9_school, 
                                    math_10_school,
                                    math_11_school,
                                    math_12_school],
                                    axis = 1)

math_scores_grade_school = math_scores_grade_school.rename(columns={0 :'9th', 1 : '10th', 2 : '11', 3 : '12'})
del math_scores_grade_school.index.name

math_scores_grade_school


,9th,10th,11,12
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


<big><b></b></big>

<big><b>Reading Score by Grade</b></big>

In [7]:
reading_9_school = students_9.groupby('school')['reading_score'].sum() / students_9_school_count
reading_10_school = students_10.groupby('school')['reading_score'].sum() / students_10_school_count
reading_11_school = students_11.groupby('school')['reading_score'].sum() / students_11_school_count
reading_12_school = students_12.groupby('school')['reading_score'].sum() / students_12_school_count

reading_scores_grade_school = pd.concat([reading_9_school, 
                                    reading_10_school,
                                    reading_11_school,
                                    reading_12_school],
                                    axis = 1)

reading_scores_grade_school = reading_scores_grade_school.rename(columns={0 :'9th', 1 : '10th', 2 : '11', 3 : '12'})
del reading_scores_grade_school.index.name

reading_scores_grade_school


,9th,10th,11,12
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


<big><b>Scores by School Spending</b></big>

In [8]:
spending_df = schools_df
spending_students_df = students_df

spending_df['Per Student Budget'] = spending_df['budget'] / spending_df['size']

bins = [0, 585, 615, 645, 675]
group_labels = ['<$585','$585-615','$615-645','$645-675']
spending_df['Spending Ranges (Per Student)'] = pd.cut(spending_df['Per Student Budget'],bins,labels=group_labels)

spending_students_df = spending_students_df.merge(spending_df, left_on='school', right_on='name')

students_by_budget = spending_students_df.groupby('Spending Ranges (Per Student)')['name_x'].count()

avg_math_score_budget = spending_students_df.groupby('Spending Ranges (Per Student)')['math_score'].sum() / students_by_budget
avg_reading_score_budget = spending_students_df.groupby('Spending Ranges (Per Student)')['reading_score'].sum() / students_by_budget

passing_math_budget = (spending_students_df.loc[spending_students_df['math_score'] > 60, :].groupby('Spending Ranges (Per Student)')['math_score'].count() / students_by_budget) * 100
passing_reading_budget = (spending_students_df.loc[spending_students_df['reading_score'] > 60, :].groupby('Spending Ranges (Per Student)')['reading_score'].count() / students_by_budget) * 100

scores_by_spending = pd.concat([avg_math_score_budget, 
                                 avg_reading_score_budget, 
                                 passing_math_budget, 
                                 passing_reading_budget], 
                               axis = 1)

scores_by_spending = scores_by_spending.rename(columns={0 : 'Average Math Score',
                                                       1 : 'Average Reading Score',
                                                       2 : '% Passing Math',
                                                       3 : '% Passing Reading'})

scores_by_spending['% Overall Passing Rate'] = (scores_by_spending['% Passing Math'] + scores_by_spending['% Passing Reading']) / 2

scores_by_spending.index.rename('Spending Ranges (Per Student)', inplace=True)

scores_by_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.363065,83.964039,100.000000,100.0,100.000000
$585-615,83.529196,83.838414,100.000000,100.0,100.000000
$615-645,78.061635,81.434088,89.209726,100.0,94.604863
$645-675,77.049297,81.005604,86.640136,100.0,93.320068


<big><b>Scores by School Size</b></big>

In [9]:
size_df = schools_df
size_students_df = students_df

size_df['Per Student Budget'] = size_df['budget'] / size_df['size']

bins = [0, 1000, 2000, 5000]
group_labels = ['Small (<1000)','Medium (1000-2000)','Large (2000-5000)']
size_df['School Sizes'] = pd.cut(size_df['size'],bins,labels=group_labels)

size_students_df = size_students_df.merge(size_df, left_on='school', right_on='name')

size_students_df.drop(['name_y'], axis=1, inplace=True)

students_by_size = size_students_df.groupby('School Sizes')['name_x'].count()

avg_math_score_size = size_students_df.groupby('School Sizes')['math_score'].sum() / students_by_size
avg_reading_score_size = size_students_df.groupby('School Sizes')['reading_score'].sum() / students_by_size

passing_math_size = (size_students_df.loc[size_students_df['math_score'] > 60, :].groupby('School Sizes')['math_score'].count() / students_by_size) * 100
passing_reading_size = (size_students_df.loc[size_students_df['reading_score'] > 60, :].groupby('School Sizes')['reading_score'].count() / students_by_size) * 100


scores_by_size = pd.concat([avg_math_score_size, 
                                 avg_reading_score_size, 
                                 passing_math_size, 
                                 passing_reading_size], 
                               axis = 1)

scores_by_size = scores_by_size.rename(columns={0 : 'Average Math Score',
                                                       1 : 'Average Reading Score',
                                                       2 : '% Passing Math',
                                                       3 : '% Passing Reading'})

scores_by_size['% Overall Passing Rate'] = (scores_by_size['% Passing Math'] + scores_by_size['% Passing Reading']) / 2

scores_by_size.index.rename('School Sizes', inplace=True)

scores_by_size


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Sizes,,,,,
Small (<1000),83.828654,83.974082,100.000000,100.0,100.000000
Medium (1000-2000),83.372682,83.867989,100.000000,100.0,100.000000
Large (2000-5000),77.477597,81.198674,87.825968,100.0,93.912984


<big><b>Scores by School Type</b></big>

In [10]:
type_students_df = size_students_df

students_by_type = type_students_df.groupby('type')['name_x'].count()

avg_math_score_type = type_students_df.groupby('type')['math_score'].sum() / students_by_type
avg_reading_score_type = type_students_df.groupby('type')['reading_score'].sum() / students_by_type

passing_math_type = (type_students_df.loc[type_students_df['math_score'] > 60, :].groupby('type')['math_score'].count() / students_by_type) * 100
passing_reading_type = (type_students_df.loc[type_students_df['reading_score'] > 60, :].groupby('type')['reading_score'].count() / students_by_type) * 100


scores_by_type = pd.concat([avg_math_score_type, 
                                 avg_reading_score_type, 
                                 passing_math_type, 
                                 passing_reading_type], 
                               axis = 1)

scores_by_type = scores_by_type.rename(columns={0 : 'Average Math Score',
                                                       1 : 'Average Reading Score',
                                                       2 : '% Passing Math',
                                                       3 : '% Passing Reading'})

scores_by_type['% Overall Passing Rate'] = (scores_by_type['% Passing Math'] + scores_by_type['% Passing Reading']) / 2

scores_by_type.index.rename('School Type', inplace=True)

scores_by_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.406183,83.902821,100.00000,100.0,100.000000
District,76.987026,80.962485,86.79567,100.0,93.397835
